In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 写一下常用的节点中心性指标

k-core：k核值，k-core算法通常用来找出一个图中符合指定k核心度的子图，该子图在图中承担着核心的地位，核心度越高，子图越小，该子图对应的核心度也越大。

In [ ]:
#k-core
#input: graph G
#output: the k-core value of each node as a dictionary
def k_core(G):
    k_core = {}
    k = 1
    min_degree_nodes=[]
    while nx.number_of_nodes(G) > 0:
        curr_min_degree_nodes = [node for node, degree in G.degree() if degree < k]
        min_degree_nodes+=curr_min_degree_nodes
        while(len(curr_min_degree_nodes)>0):
            G.remove_nodes_from(curr_min_degree_nodes)
            curr_min_degree_nodes=[node for node,degree in G.degree() if degree<k]
            min_degree_nodes+=curr_min_degree_nodes
        
        for node in min_degree_nodes:
            k_core[node] = k-1

        min_degree_nodes.clear()
        
        k += 1
    
    return k_core

networkx自带的一些中心性指标，可以在这个链接查看

https://networkx.org/documentation/stable/reference/algorithms/centrality.html

#### Collective Influence
CI算法，将每个节点的影响力视作和自己邻近的一部分节点的影响力的累积，对于节点i，其对应的CI值为：
$$CI_i=(k_i-1)\sum_{j\in\delta(Ball,l)}(k_j-1)$$
这里的参数代表：$\delta(Ball,l)$代表以目标节点i为中心，可达半径为l的球面上面的节点的集合

In [ ]:
#input: graph G
#algorithm: the CI value of node v is the sum of the degree of the neighbors of node v, with their distance to node v = l
#output: CI(G)
def CI(G,l):
    CI={}
    if(l==0):
        for node in G.nodes():
            CI[node]=(nx.degree(G,node)-1)**2
        return CI
    radius=l
    for node in G.nodes():
        CI[node]=0

    for node in G.nodes():
        i=1
        current_nbrs=nx.all_neighbors(G,node)
        final_nbrs={}
        #find the nodes on the radius l,the node distance < l not include, of node v
        while i<=radius:
            next_nbrs=[]
            for nbr in current_nbrs:
                next_nbrs.extend(nx.all_neighbors(G,nbr))
            final_nbrs=set(next_nbrs)-set(current_nbrs)
            #if node in final_nbrs, remove it
            if node in final_nbrs:
                final_nbrs.remove(node)
            current_nbrs=next_nbrs
            i+=1
        
        sum=0
        for nbr in final_nbrs:
            sum+=nx.degree(G,nbr)-1

        CI[node]=(nx.degree(G,node)-1)*sum

    return CI

#### Coreness Centrality
将每个节点的KC值表示为节点的邻居的k-shell值的组合，因此，对于节点v，有：
$$CC(v)=\sum_{w\in nbr(v)}ks(w)$$
其中：$nbr(v)$代表节点v的邻居集合，$ks(w)$代表节点w的$k-shell$值

In [ ]:
#input:Graph G
#algorithm:the coreness centrality of node v is the sum of the k-shell values of its neighbors
#output:coreness centrality of each node
def coreness_centrality(G):
    #get the k-core of G
    k_core = nx.core_number(G)
    k_shell={}
    CC={}
    for node in G.nodes():
        CC[node]=k_core[node]
        k_shell[node]=k_core[node]
    maxCore=max(k_core.values())

    for i in range(maxCore):
        shellG=nx.k_shell(G,i)
        for node in shellG.nodes():
            k_shell[node]=i

    for node in G.nodes():
        nodecc=0
        nbrs=nx.all_neighbors(G,node)
        for nbr in nbrs:
            nodecc+=k_shell[nbr]
        CC[node]=nodecc

    return CC

扩展的核心重要性，将节点v的CC值表示为：
$$CC(v)=\sum_{w\in nbrs(v)}CC(w)$$
其中，$nbrs(v)$代表节点v的邻居集合，$CC(w)$代表节点w的CC值

In [ ]:
#input:Graph G
#algorithm:the extend coreness centrality of node v is the sum of the coreness centrality values of its neighbors
#output:extend coreness centrality of each node
def extend_CC(G):
    #get the k-core of G
    k_core = nx.core_number(G)
    k_shell={}
    CC={}
    eCC={}
    for node in G.nodes():
        CC[node]=k_core[node]
        k_shell[node]=k_core[node]
        eCC[node]=k_core[node]
    maxCore=max(k_core.values())

    for i in range(maxCore):
        shellG=nx.k_shell(G,i)
        for node in shellG.nodes():
            k_shell[node]=i

    for node in G.nodes():
        nodecc=0
        nbrs=nx.all_neighbors(G,node)
        for nbr in nbrs:
            nodecc+=k_shell[nbr]
        CC[node]=nodecc

    for node in G.nodes():
        node_ecc=0
        nbrs=nx.all_neighbors(G,node)
        for nbr in nbrs:
            node_ecc+=CC[nbr]
        eCC[node]=node_ecc

    return eCC

CR，NC